In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
import torchvision
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from ptflops import get_model_complexity_info

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

np.random.seed(42)
torch.manual_seed(42)
width_mult_list = [0.25, 0.5, 0.75, 1.0]

def make_divisible(v, divisor=8, min_value=1):
    """
    forked from slim:
    https://github.com/tensorflow/models/blob/\
    0344c5503ee55e24f0de7f37336a6e08f10976fd/\
    research/slim/nets/mobilenet/mobilenet.py#L62-L69
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


class SlimmableConv2d(nn.Conv2d):
    def __init__(self, in_channels_list, out_channels_list,
                 kernel_size, stride=1, padding=0, dilation=1,
                 groups_list=[1], bias=True):
        super(SlimmableConv2d, self).__init__(
            max(in_channels_list), max(out_channels_list),
            kernel_size, stride=stride, padding=padding, dilation=dilation,
            groups=max(groups_list), bias=bias)
        self.in_channels_list = in_channels_list
        self.out_channels_list = out_channels_list
        self.groups_list = groups_list
        if self.groups_list == [1]:
            self.groups_list = [1 for _ in range(len(in_channels_list))]
        self.width_mult = max(width_mult_list)

    def forward(self, input):
        idx = width_mult_list.index(self.width_mult)
        self.in_channels = self.in_channels_list[idx]
        self.out_channels = self.out_channels_list[idx]
        self.groups = self.groups_list[idx]
        weight = self.weight[:self.out_channels, :self.in_channels, :, :]
        if self.bias is not None:
            bias = self.bias[:self.out_channels]
        else:
            bias = self.bias
        y = nn.functional.conv2d(
            input, weight, bias, self.stride, self.padding,
            self.dilation, self.groups)
        return y


class SlimmableLinear(nn.Linear):
    def __init__(self, in_features_list, out_features_list, bias=True):
        super(SlimmableLinear, self).__init__(
            max(in_features_list), max(out_features_list), bias=bias)
        self.in_features_list = in_features_list
        self.out_features_list = out_features_list
        self.width_mult = max(width_mult_list)

    def forward(self, input):
        idx = width_mult_list.index(self.width_mult)
        self.in_features = self.in_features_list[idx]
        self.out_features = self.out_features_list[idx]
        weight = self.weight[:self.out_features, :self.in_features]
        if self.bias is not None:
            bias = self.bias[:self.out_features]
        else:
            bias = self.bias
        return nn.functional.linear(input, weight, bias)


class Baseline(nn.Module):
    def __init__(self, width_mult):
        super(Baseline, self).__init__()
        out_channels_1 = [int(64 * width_mult) for width_mult in width_mult_list]
        #3 input channels for conv1 because input is rgb image
        self.conv1 = SlimmableConv2d([3 for _ in range(len(width_mult_list))], out_channels_1, kernel_size=3, padding="valid",bias= False)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))

        out_channels_2 = [ int(128 * width_mult) for width_mult in width_mult_list]
        self.conv2 = SlimmableConv2d(out_channels_1, out_channels_2, kernel_size=3, padding="valid",bias= False)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        out_channels_3 = [ int(64 * width_mult) for width_mult in width_mult_list]
        self.conv3 = SlimmableConv2d(out_channels_2, out_channels_3, kernel_size=3, padding="valid",bias= False)
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))

        #b,64,2,2
        self.flatten = nn.Flatten()
        #b,64x4
        out_channels_fc1 = [ int(256 * width_mult) for width_mult in width_mult_list]
        self.fc1 = SlimmableLinear([i*4 for i in out_channels_3], out_channels_fc1)

        out_channels_fc2 = [ int(64 * width_mult) for width_mult in width_mult_list]
        self.fc2 = SlimmableLinear(out_channels_fc1, out_channels_fc2)

        out_channels_fc3 = [10 for _ in range(len(width_mult_list))]
        self.fc3 = SlimmableLinear(out_channels_fc2, out_channels_fc3)
        
    def forward(self, x):
        exit_outputs = []
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        exit_outputs.append(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        exit_outputs.append(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool3(x)
        exit_outputs.append(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x, exit_outputs

model = Baseline(width_mult=0.25).to(device)
flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
print(f"\nFLOPs: {flops}")
print(f"Parameters: {params}")

model = Baseline(width_mult=0.5).to(device)
flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
print(f"\nFLOPs: {flops}")
print(f"Parameters: {params}")

model = Baseline(width_mult=0.75).to(device)
flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
print(f"\nFLOPs: {flops}")
print(f"Parameters: {params}")

model = Baseline(width_mult=1).to(device)
flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=True, print_per_layer_stat=True)
print(f"\nFLOPs: {flops}")
print(f"Parameters: {params}")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f78b24f89a0>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f783ceac220, raw_cell="import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
i.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://ssh-remote%2Blaboe/home/mal/DScale/freeml/FreeML/slimmable/slimmable.ipynb#W0sdnNjb2RlLXJlbW90ZQ%3D%3D>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Baseline(
  0, 0.000% Params, 80.26 KMac, 33.333% MACs, 
  (conv1): SlimmableConv2d(0, 0.000% Params, 0.0 Mac, 0.000% MACs, 3, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid, bias=False)
  (pool1): MaxPool2d(0, 0.000% Params, 57.6 KMac, 23.923% MACs, kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): SlimmableConv2d(0, 0.000% Params, 0.0 Mac, 0.000% MACs, 64, 128, kernel_size=(3, 3), stride=(1, 1), padding=valid, bias=False)
  (pool2): MaxPool2d(0, 0.000% Params, 21.63 KMac, 8.985% MACs, kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): SlimmableConv2d(0, 0.000% Params, 0.0 Mac, 0.000% MACs, 128, 64, kernel_size=(3, 3), stride=(1, 1), padding=valid, bias=False)
  (pool3): MaxPool2d(0, 0.000% Params, 1.02 KMac, 0.425% MACs, kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(0, 0.000% Params, 0.0 Mac, 0.000% MACs, start_dim=1, end_dim=-1)
  (fc1): SlimmableLinear(0, 0

TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given